In [7]:
import os
destpath = 'E:/imeunu/ASMR/train/train/audio/'
save_path = 'E:/imeunu/ASMR/train/train/result/'

In [8]:
import os
import re
import csv
import librosa
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras import optimizers,regularizers
from tensorflow.keras.models import Sequential, load_model, model_from_json 
from tensorflow.keras.layers import Input,Conv2D,MaxPooling2D,Dense,Activation,Flatten,GlobalAveragePooling2D,GlobalMaxPooling2D,Dropout,AveragePooling2D,BatchNormalization
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint
from tensorflow.python.framework.ops import disable_eager_execution
from sklearn.metrics import accuracy_score
from tensorflow.python.client import device_lib

#Generate training, test set
def walk(folder):
    '''Walk through every files in a directory'''
    for dirpath, dirs, files in os.walk(folder):
        for filename in files:
            yield dirpath, filename

def iscsv(ext):
    csvlist = ['.csv']
    if ext in csvlist:
        return True
    else:
        return False

def find_word(path):
    keywords=[]
    for folder, filename in walk(path):
        keyword = re.split('/',folder)[-1]
        if keyword not in keywords:
            keywords.append(keyword)
    return tuple(keywords)
  
def count_ext(path, ext):
    count = 0
    for folder, filename in walk(path):
        if filename[-len(ext):] == ext:
            count += 1
    return count

In [41]:
try:
    os.mkdir(save_path)
except:
    pass
keywords = find_word(destpath)
print(keywords)

X_data = [];Y_data = []
from tqdm import tqdm
total = count_ext(destpath, '.wav')
with tqdm(total=total) as pbar:
    for folder, filename in walk(destpath):
        temp = []
        ext = os.path.splitext(filename)[-1]
#         if not iscsv(ext):
#             continue
        sample, sr = librosa.load(folder+'/'+filename,sr=16000)
        sample = librosa.util.fix_length(sample,16000)
        mfcc = librosa.feature.mfcc(sample,n_fft=2048,hop_length=512,n_mfcc=14)
        temp.append(mfcc)
        temp.append(librosa.feature.delta(mfcc,order=1))
        temp.append(librosa.feature.delta(mfcc,order=2))
        temp = np.array(temp).reshape((32,14,3))
        X_data.append(temp)
        Y_data.append(keywords.index(re.split('/',folder)[-1]))
        pbar.update(1)
Y_data = np.expand_dims(Y_data, axis=-1)
print('X_data shape:',np.shape(X_data))
print('Y_data shape:',np.shape(Y_data))
print('X_data len : %d, Y_data len : %d' % (len(X_data), len(Y_data)))

  0%|          | 0/64721 [00:00<?, ?it/s]

('bed', 'bird', 'cat', 'dog', 'down', 'eight', 'five', 'four', 'go', 'happy', 'house', 'left', 'marvin', 'nine', 'no', 'off', 'on', 'one', 'right', 'seven', 'sheila', 'six', 'stop', 'three', 'tree', 'two', 'up', 'wow', 'yes', 'zero')


100%|██████████| 64721/64721 [13:24<00:00, 80.47it/s] 


X_data shape: (64721, 32, 14, 3)
Y_data shape: (64721, 1)
X_data len : 64721, Y_data len : 64721


In [46]:
from sklearn.model_selection import train_test_split
train_X, test_X, train_Y, test_Y= train_test_split(X_data, Y_data, test_size= 0.2, stratify= Y_data)
from keras.utils import to_categorical
train_X= np.array(train_X)
test_X= np.array(test_X)
train_Y= np.array(train_Y)
test_Y= np.array(test_Y)
train_Y = to_categorical(train_Y, len(keywords))
test_Y = to_categorical(test_Y, len(keywords))

In [47]:
#Model
inp_shape = train_X.shape[1:]
def generate_model_cnn(mode='words', pooling=0, layers=3, kernel=11):
    model = Sequential()
    print("=================================CNN=====================================")
    model.add(Conv2D(filters=30, kernel_size=(3,3), strides = 1, padding ='same', input_shape=inp_shape, activation = 'relu')) #add convolution layer
    model.add(AveragePooling2D(pool_size=(3,3), strides=2)) #pooling layer
    model.add(Conv2D(filters=30, kernel_size=(3,3), strides = 1, padding ='same', activation = 'relu'))
    model.add(AveragePooling2D(pool_size=(3,3), strides=None))
    model.add(Flatten())
    model.add(Dense(50, activation = 'relu',kernel_regularizer=regularizers.l2(0.001)))
    model.add(Dropout(0.1))
    model.add(Dense(30, activation = 'softmax'))
    return model

def generate_model(mode='words', summary=True):
    model = generate_model_cnn(mode=mode, pooling=0)
    if summary:
        model.summary()
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

callbacks_list  = [
    EarlyStopping( monitor = 'loss',  min_delta=0.001, patience=100,
                  verbose=1, mode='auto'),
    ModelCheckpoint(filepath = save_path+"/weights.{epoch:02d}-{val_loss:.3f}-{val_accuracy:.2f}.hdf5",
                    monitor = 'val_loss',
                    save_best_only=True ),
    ReduceLROnPlateau(monitor='val_acc', factor=0.1, patience=7, verbose=1, min_delta=1e-4)]

model = generate_model(mode='words',summary=True)
history = model.fit(train_X, train_Y, epochs=300, batch_size=100, validation_data=(test_X,test_Y),callbacks=callbacks_list)

=================================CNN=====================================
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 32, 14, 30)        840       
_________________________________________________________________
average_pooling2d (AveragePo (None, 15, 6, 30)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 15, 6, 30)         8130      
_________________________________________________________________
average_pooling2d_1 (Average (None, 5, 2, 30)          0         
_________________________________________________________________
flatten (Flatten)            (None, 300)               0         
_________________________________________________________________
dense (Dense)                (None, 50)                15050     
________________________________________________

Epoch 23/300
518/518 [==============================] - 22s 43ms/step - loss: 0.8906 - accuracy: 0.7409 - val_loss: 0.8121 - val_accuracy: 0.7738
Epoch 24/300
518/518 [==============================] - 22s 42ms/step - loss: 0.8746 - accuracy: 0.7489 - val_loss: 0.7852 - val_accuracy: 0.7800
Epoch 25/300
518/518 [==============================] - 23s 45ms/step - loss: 0.8636 - accuracy: 0.7478 - val_loss: 0.7793 - val_accuracy: 0.7835
Epoch 26/300
518/518 [==============================] - 24s 46ms/step - loss: 0.8624 - accuracy: 0.7494 - val_loss: 0.7906 - val_accuracy: 0.7828
Epoch 27/300
518/518 [==============================] - 22s 43ms/step - loss: 0.8665 - accuracy: 0.7510 - val_loss: 0.8233 - val_accuracy: 0.7753
Epoch 28/300
518/518 [==============================] - 22s 43ms/step - loss: 0.8576 - accuracy: 0.7514 - val_loss: 0.8014 - val_accuracy: 0.7746
Epoch 29/300
518/518 [==============================] - 21s 41ms/step - loss: 0.8501 - accuracy: 0.7521 - val_loss: 0.7903 -

518/518 [==============================] - 21s 40ms/step - loss: 0.7420 - accuracy: 0.7827 - val_loss: 0.7340 - val_accuracy: 0.7988
Epoch 50/300
518/518 [==============================] - 21s 41ms/step - loss: 0.7427 - accuracy: 0.7842 - val_loss: 0.7573 - val_accuracy: 0.7929
Epoch 51/300
518/518 [==============================] - 22s 42ms/step - loss: 0.7378 - accuracy: 0.7847 - val_loss: 0.7080 - val_accuracy: 0.8080
Epoch 52/300
518/518 [==============================] - 21s 40ms/step - loss: 0.7292 - accuracy: 0.7838 - val_loss: 0.7728 - val_accuracy: 0.7881
Epoch 53/300
518/518 [==============================] - 22s 42ms/step - loss: 0.7371 - accuracy: 0.7827 - val_loss: 0.7387 - val_accuracy: 0.7968
Epoch 54/300
518/518 [==============================] - 22s 42ms/step - loss: 0.7414 - accuracy: 0.7832 - val_loss: 0.7770 - val_accuracy: 0.7823
Epoch 55/300
518/518 [==============================] - 21s 41ms/step - loss: 0.7233 - accuracy: 0.7894 - val_loss: 0.7121 - val_accuracy

Epoch 76/300
518/518 [==============================] - 24s 47ms/step - loss: 0.6870 - accuracy: 0.7978 - val_loss: 0.7108 - val_accuracy: 0.8091
Epoch 77/300
518/518 [==============================] - 21s 41ms/step - loss: 0.6762 - accuracy: 0.8043 - val_loss: 0.7072 - val_accuracy: 0.8079
Epoch 78/300
518/518 [==============================] - 20s 38ms/step - loss: 0.6759 - accuracy: 0.8001 - val_loss: 0.7031 - val_accuracy: 0.8058
Epoch 79/300
518/518 [==============================] - 22s 42ms/step - loss: 0.6877 - accuracy: 0.7978 - val_loss: 0.6979 - val_accuracy: 0.8070
Epoch 80/300
518/518 [==============================] - 22s 43ms/step - loss: 0.6889 - accuracy: 0.7980 - val_loss: 0.7016 - val_accuracy: 0.8059
Epoch 81/300
518/518 [==============================] - 24s 46ms/step - loss: 0.6863 - accuracy: 0.7984 - val_loss: 0.6804 - val_accuracy: 0.8139
Epoch 82/300
518/518 [==============================] - 20s 39ms/step - loss: 0.6716 - accuracy: 0.8028 - val_loss: 0.7040 -

Epoch 102/300
518/518 [==============================] - 19s 37ms/step - loss: 0.6465 - accuracy: 0.8097 - val_loss: 0.6873 - val_accuracy: 0.8132
Epoch 103/300
518/518 [==============================] - 19s 36ms/step - loss: 0.6449 - accuracy: 0.8081 - val_loss: 0.6786 - val_accuracy: 0.8189
Epoch 104/300
518/518 [==============================] - 18s 35ms/step - loss: 0.6305 - accuracy: 0.8130 - val_loss: 0.7205 - val_accuracy: 0.8042
Epoch 105/300
518/518 [==============================] - 18s 35ms/step - loss: 0.6550 - accuracy: 0.8073 - val_loss: 0.6829 - val_accuracy: 0.8140
Epoch 106/300
518/518 [==============================] - 18s 35ms/step - loss: 0.6443 - accuracy: 0.8106 - val_loss: 0.6833 - val_accuracy: 0.8114
Epoch 107/300
518/518 [==============================] - 19s 37ms/step - loss: 0.6367 - accuracy: 0.8149 - val_loss: 0.7044 - val_accuracy: 0.8077
Epoch 108/300
518/518 [==============================] - 18s 35ms/step - loss: 0.6410 - accuracy: 0.8123 - val_loss: 0

...

Epoch 181/300
518/518 [==============================] - 15s 29ms/step - loss: 0.5830 - accuracy: 0.8277 - val_loss: 0.6735 - val_accuracy: 0.8195
Epoch 182/300
518/518 [==============================] - 15s 28ms/step - loss: 0.5891 - accuracy: 0.8232 - val_loss: 0.6662 - val_accuracy: 0.8168
Epoch 183/300
518/518 [==============================] - 14s 27ms/step - loss: 0.5808 - accuracy: 0.8285 - val_loss: 0.6603 - val_accuracy: 0.8212
Epoch 184/300
518/518 [==============================] - 14s 27ms/step - loss: 0.5847 - accuracy: 0.8256 - val_loss: 0.7388 - val_accuracy: 0.8026
Epoch 185/300
518/518 [==============================] - 14s 27ms/step - loss: 0.5859 - accuracy: 0.8253 - val_loss: 0.7041 - val_accuracy: 0.8103
Epoch 186/300
518/518 [==============================] - 14s 27ms/step - loss: 0.5924 - accuracy: 0.8229 - val_loss: 0.6631 - val_accuracy: 0.8203
Epoch 187/300
518/518 [==============================] - 15s 29ms/step - loss: 0.5797 - accuracy: 0.8284 - val_loss: 0

Epoch 208/300
518/518 [==============================] - 15s 30ms/step - loss: 0.5847 - accuracy: 0.8238 - val_loss: 0.6641 - val_accuracy: 0.8202
Epoch 209/300
518/518 [==============================] - 14s 28ms/step - loss: 0.5814 - accuracy: 0.8289 - val_loss: 0.7048 - val_accuracy: 0.8082
Epoch 210/300
518/518 [==============================] - 15s 28ms/step - loss: 0.5821 - accuracy: 0.8278 - val_loss: 0.6689 - val_accuracy: 0.8225
Epoch 211/300
518/518 [==============================] - 15s 29ms/step - loss: 0.5702 - accuracy: 0.8315 - val_loss: 0.6517 - val_accuracy: 0.8248
Epoch 212/300
518/518 [==============================] - 15s 28ms/step - loss: 0.5659 - accuracy: 0.8321 - val_loss: 0.6852 - val_accuracy: 0.8154
Epoch 213/300
518/518 [==============================] - 14s 27ms/step - loss: 0.5842 - accuracy: 0.8284 - val_loss: 0.6645 - val_accuracy: 0.8195
Epoch 214/300
518/518 [==============================] - 15s 29ms/step - loss: 0.5703 - accuracy: 0.8304 - val_loss: 0

Epoch 234/300
518/518 [==============================] - 15s 28ms/step - loss: 0.5689 - accuracy: 0.8310 - val_loss: 0.7040 - val_accuracy: 0.8093
Epoch 235/300
518/518 [==============================] - 15s 28ms/step - loss: 0.5740 - accuracy: 0.8284 - val_loss: 0.6565 - val_accuracy: 0.8257
Epoch 236/300
518/518 [==============================] - 15s 29ms/step - loss: 0.5702 - accuracy: 0.8325 - val_loss: 0.6572 - val_accuracy: 0.8222
Epoch 237/300
518/518 [==============================] - 16s 30ms/step - loss: 0.5656 - accuracy: 0.8345 - val_loss: 0.6787 - val_accuracy: 0.8131
Epoch 238/300
518/518 [==============================] - 15s 30ms/step - loss: 0.5669 - accuracy: 0.8311 - val_loss: 0.6606 - val_accuracy: 0.8233
Epoch 239/300
518/518 [==============================] - 18s 35ms/step - loss: 0.5645 - accuracy: 0.8319 - val_loss: 0.6616 - val_accuracy: 0.8185
Epoch 240/300
518/518 [==============================] - 18s 35ms/step - loss: 0.5609 - accuracy: 0.8356 - val_loss: 0

Epoch 261/300
518/518 [==============================] - 16s 31ms/step - loss: 0.5590 - accuracy: 0.8341 - val_loss: 0.6880 - val_accuracy: 0.8173
Epoch 262/300
518/518 [==============================] - 15s 30ms/step - loss: 0.5646 - accuracy: 0.8304 - val_loss: 0.6543 - val_accuracy: 0.8237
Epoch 263/300
518/518 [==============================] - 15s 29ms/step - loss: 0.5569 - accuracy: 0.8330 - val_loss: 0.6710 - val_accuracy: 0.8150
Epoch 264/300
518/518 [==============================] - 15s 29ms/step - loss: 0.5617 - accuracy: 0.8313 - val_loss: 0.7068 - val_accuracy: 0.8082
Epoch 265/300
518/518 [==============================] - 15s 29ms/step - loss: 0.5446 - accuracy: 0.8372 - val_loss: 0.6721 - val_accuracy: 0.8162
Epoch 266/300
518/518 [==============================] - 15s 29ms/step - loss: 0.5705 - accuracy: 0.8295 - val_loss: 0.6924 - val_accuracy: 0.8109
Epoch 267/300
518/518 [==============================] - 15s 28ms/step - loss: 0.5574 - accuracy: 0.8336 - val_loss: 0

Epoch 287/300
518/518 [==============================] - 15s 29ms/step - loss: 0.5613 - accuracy: 0.8321 - val_loss: 0.6943 - val_accuracy: 0.8114
Epoch 288/300
518/518 [==============================] - 15s 29ms/step - loss: 0.5544 - accuracy: 0.8344 - val_loss: 0.6632 - val_accuracy: 0.8228
Epoch 289/300
518/518 [==============================] - 15s 29ms/step - loss: 0.5452 - accuracy: 0.8372 - val_loss: 0.6762 - val_accuracy: 0.8192
Epoch 290/300
518/518 [==============================] - 15s 29ms/step - loss: 0.5437 - accuracy: 0.8366 - val_loss: 0.6634 - val_accuracy: 0.8205
Epoch 291/300
518/518 [==============================] - 16s 31ms/step - loss: 0.5445 - accuracy: 0.8369 - val_loss: 0.6959 - val_accuracy: 0.8154
Epoch 292/300
518/518 [==============================] - 15s 30ms/step - loss: 0.5532 - accuracy: 0.8334 - val_loss: 0.6543 - val_accuracy: 0.8262
Epoch 293/300
518/518 [==============================] - 15s 29ms/step - loss: 0.5457 - accuracy: 0.8360 - val_loss: 0

In [50]:
def save_history_plot(hist,path):
    plt.ioff()
    fig, loss_ax = plt.subplots()
    acc_ax = loss_ax.twinx()
    loss_ax.plot(hist.history['loss'], 'y', label='train loss')
    loss_ax.plot(hist.history['val_loss'], 'r', label='val loss')
    loss_ax.set_xlabel('epoch')
    loss_ax.set_ylabel('loss')
    loss_ax.legend(loc='upper left')
    acc_ax.plot(hist.history['accuracy'], 'b', label='accuracy')
    acc_ax.plot(hist.history['val_accuracy'], 'g', label='val accuracy')
    if 'top_k_categorical_accuracy' in hist.history:
        acc_ax.plot(hist.history['val_top_k_categorical_accuracy'], label='val top_k_accuracy')
        acc_ax.plot(hist.history['top_k_categorical_accuracy'], label='top_k_accuracy')
    acc_ax.set_ylabel('accuracy')
    acc_ax.legend(loc='upper right')
    plt.savefig(path)
    # plt.show()
    plt.close('all')
save_history_plot(history, save_path+"/model_history_img.png")
pd.DataFrame(history.history).to_csv(save_path+"/history.csv")

print("-- Evaluate --")
scores = model.evaluate(test_X, test_Y, batch_size=100)
print("%s: %.2f%%" %(model.metrics_names[0], scores[0]))
print("%s: %.2f%%" %(model.metrics_names[1], scores[1]*100))

print("-- Predict --")
#output = model.predict_generator(my_test_batch_generator, workers = workers, use_multiprocessing = True)
output = model.predict(test_X)
np.set_printoptions(formatter={'float': lambda x: "{0:0.3f}".format(x)})

tt=[]
for i in range(len(output)):
    temp=[]
    temp.append(output[i].argsort()[::-1][:5] )
    temp.append(np.round(np.sort(output[i])[::-1][:5],6))
    tt.append(temp)
predict_df = pd.DataFrame(tt)

predicted_classes= output.argmax(axis=1)
answer_classes= test_Y.argmax(axis=1)
acc = accuracy_score(answer_classes, predicted_classes)
print('test accuracy :', acc)

# model save
model_json = model.to_json()
with open(save_path+"/model_acc_{:.4f}.json".format(acc), 'w') as json_file:
    json_file.write(model_json)

# weight save
model.save_weights(save_path +"/model_weight.h5")
model_json = model.to_json()
with open(save_path+"/model.json".format(acc), 'w') as json_file:
    json_file.write(model_json)

-- Evaluate --
130/130 [==============================] - 1s 8ms/step - loss: 0.6720 - accuracy: 0.8204
loss: 0.67%
accuracy: 82.04%
-- Predict --
test accuracy : 0.8203939745075318
